# Johnnie Worker

In [50]:
from coding import *

In [51]:
x_=pd.read_csv("c:/code/x.csv")
x_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448 entries, 0 to 1447
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   idx                       1448 non-null   int64 
 1   trainingSemester          1448 non-null   object
 2   trainingClass             1448 non-null   object
 3   trainingSerie             1448 non-null   object
 4   trainingGroup             1448 non-null   object
 5   trainingCompany           1448 non-null   object
 6   trainingPeriod            1448 non-null   object
 7   name                      1448 non-null   object
 8   registerProblem           1448 non-null   object
 9   trainingTeacher           1448 non-null   object
 10  trainingUnit              1448 non-null   object
 11  trainingLeader            1448 non-null   object
 12  trainingLeaderDepartment  1448 non-null   object
 13  trainingLeaderPosition    1448 non-null   object
 14  trainingLeaderDegree    

In [52]:
x=kabone(x_)
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1304 entries, 0 to 1446
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   학번                      1304 non-null   int64  
 1   성명                      1304 non-null   object 
 2   성별                      0 non-null      float64
 3   학년                      1304 non-null   int64  
 4   학기상세                    1304 non-null   object 
 5   연도                      1304 non-null   object 
 6   학기                      1304 non-null   object 
 7   교과목명                    1304 non-null   object 
 8   학점                      1304 non-null   int64  
 9   시수                      1304 non-null   int64  
 10  과목명
학년
(학점x시수x주수=실습시간)  1304 non-null   object 
 11  실습기관명                   1304 non-null   object 
 12  실습병동(진료과)               1304 non-null   object 
 13  교과목 담당교원(전임)            1005 non-null   object 
 14  교과목 담당교원(비전임)           299 non-null    objec

In [53]:
x=x.iloc[:,[5,6,10,11,12,13,14]]

In [54]:
trGroupUnitIndices=["trainingSemester","trainingClass","trainingCompany","trainingUnit"]
trGroupUnit=x_.query("trainingSemester!='3-2.5-2023'").groupby(trGroupUnitIndices).trainingGroup.transform("nunique").rename("trGroupUnit")
trGroupUnit.shape[0]==1304

True

In [55]:
trIdxCountPerUnit=x_.query("trainingSemester!='3-2.5-2023'").groupby(trGroupUnitIndices).idx.transform("nunique").rename("trIdxCountPerUnit")
len(trIdxCountPerUnit)==1304

True

In [56]:
trPeriod=x_.query("~(trainingSemester.str.contains('3-2.5-2023'))").trainingPeriod
trPeriod.shape[0]==1304

True

In [57]:
final_=pd.concat([x,trGroupUnit,trIdxCountPerUnit,trPeriod],axis=1)
final_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1304 entries, 0 to 1446
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   연도                      1304 non-null   object
 1   학기                      1304 non-null   object
 2   과목명
학년
(학점x시수x주수=실습시간)  1304 non-null   object
 3   실습기관명                   1304 non-null   object
 4   실습병동(진료과)               1304 non-null   object
 5   교과목 담당교원(전임)            1005 non-null   object
 6   교과목 담당교원(비전임)           299 non-null    object
 7   trGroupUnit             1304 non-null   int64 
 8   trIdxCountPerUnit       1304 non-null   int64 
 9   trainingPeriod          1304 non-null   object
dtypes: int64(2), object(8)
memory usage: 112.1+ KB


In [58]:
final=final_.drop_duplicates().sort_values(list(final_.columns[:-2])).assign(gye=np.nan)
final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 624 entries, 507 to 612
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   연도                      624 non-null    object 
 1   학기                      624 non-null    object 
 2   과목명
학년
(학점x시수x주수=실습시간)  624 non-null    object 
 3   실습기관명                   624 non-null    object 
 4   실습병동(진료과)               624 non-null    object 
 5   교과목 담당교원(전임)            493 non-null    object 
 6   교과목 담당교원(비전임)           131 non-null    object 
 7   trGroupUnit             624 non-null    int64  
 8   trIdxCountPerUnit       624 non-null    int64  
 9   trainingPeriod          624 non-null    object 
 10  gye                     0 non-null      float64
dtypes: float64(1), int64(2), object(8)
memory usage: 58.5+ KB


## Validation

In [59]:
final.iloc[[379,378]]

,연도,학기,과목명\n학년\n(학점x시수x주수=실습시간),실습기관명,실습병동(진료과),교과목 담당교원(전임),교과목 담당교원(비전임),trGroupUnit,trIdxCountPerUnit,trainingPeriod,gye
768,2023,2,성인간호학실습2\n(3학년)\n81,대전을지대학교병원,15B(소화기내과),김진,NaN,1,6,2023. 11. 06. ~ 2023. 11. 24.,NaN
750,2023,2,성인간호학실습2\n(3학년)\n81,대전을지대학교병원,10A(정형외과),김진,NaN,1,6,2023. 11. 27. ~ 2023. 12. 15.,NaN


In [60]:
kotae=x_.query("(trainingSemester=='3-2.0-2023') & (trainingCompany.str.contains('대전을',na=False))")
print(f"{kotae.trainingClass.unique()=}\n{kotae.trainingUnit.unique()=}\n{kotae.trainingPeriod.unique()=}\n{kotae.groupby('trainingUnit').idx.nunique()=}")

kotae.trainingClass.unique()=array(['성인간호학실습2'], dtype=object)
kotae.trainingUnit.unique()=array(['10A', '15B'], dtype=object)
kotae.trainingPeriod.unique()=array(['2023. 11. 06. ~ 2023. 11. 24.', '2023. 11. 27. ~ 2023. 12. 15.'],
      dtype=object)
kotae.groupby('trainingUnit').idx.nunique()=trainingUnit
10A    6
15B    6
Name: idx, dtype: int64


* trGroupUnit은 많이 나오는 경우가 없음: 보통 1개 trainingGroup이 1개 trainingUnit 감